# [Module 1] Train a Keras Sequential Model (TensorFlow 2.1)

### [Note] 본 주피터 노트북은 TensorFlow 2.1에서 핸즈온을 수행합니다. Amazon SageMaker는 2020년 1월부터 빌트인 딥러닝 컨테이너 형태로 TensorFlow 2.0을 지원하고 있습니다.

본 노트북(notebook)은 SageMaker 상에서 Keras Sequential model을 학습하는 방법을 단계별로 설명합니다. 본 노트북에서 사용한 모델은 간단한 deep CNN(Convolutional Neural Network) 모델로 [the Keras examples](https://github.com/keras-team/keras/blob/master/examples/cifar10_cnn.py)에 소개된 모델과 동일합니다.
- 참고로, 본 모델은 25 epoch 학습 후에 검증셋의 정확도(accuracy)가 약 75%이고 50 epoch 학습 후에 검증셋의 정확도가 약 79% 입니다.
- 본 워크샵 과정에서는 시간 관계상 5 epoch까지만 학습합니다. (단, Horovod 기반 분산 학습은 10 epoch까지 학습합니다.)

In [1]:
import tensorflow as tf
import numpy as np
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

tf.__version__

Num GPUs Available:  0


'2.1.0'

## Getting the data
아래 코드를 실행하여 Cifar dataset을 다운로드 받고 TFRecord데이터로 변환합니다. 

In [ ]:
#"generate_cifar10_tfrecords_v2.py"가 궁금할 경우 아래 #을 제거 하고 실행합니다.
#!cat generate_cifar10_tfrecords_v2.py

In [2]:
!python generate_cifar10_tfrecords_v2.py --data-dir ./data

Download from https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz and extract.
./data
170500096/170498071 [==============================] - 2s 0us/step
Generating ./data/train/train.tfrecords
Generating ./data/validation/validation.tfrecords
Generating ./data/eval/eval.tfrecords
Done!


## Run the training locally

본 스크립트는 모델 학습에 필요한 인자값(arguments)들을 사용합니다. 모델 학습에 필요한 인자값들은 아래와 같습니다.

1. `model_dir` - 로그와 체크 포인트를 저장하는 경로
2. `train, validation, eval` - TFRecord 데이터셋을 저장하는 경로
3. `epochs` - epoch 횟수

아래 명령어로 **<font color='red'>SageMaker 관련 API 호출 없이</font>** 로컬 노트북 인스턴스 환경에서 1 epoch만 학습해 봅니다. 참고로, MacBook Pro(15-inch, 2018) 2.6GHz Core i7 16GB 사양에서 2분 20초~2분 40초 소요됩니다.

In [3]:
%%time
!mkdir -p logs
!python training_script/cifar10_keras_tf2.py --model_dir ./logs \
                                         --train data/train \
                                         --validation data/validation \
                                         --eval data/eval \
                                         --epochs 1
!rm -rf logs

2020-09-07 12:17:01.967343: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX512F
2020-09-07 12:17:01.992230: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2999995000 Hz
2020-09-07 12:17:01.992619: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x5604cd5e4e30 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-09-07 12:17:01.992635: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2020-09-07 12:17:01.992722: I tensorflow/core/common_runtime/process_util.cc:147] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
Train for 312 steps, validate for 78 steps
312/312 [==============================] - 49s 157ms/step - loss: 1.8737 - accuracy: 0.3144 - val_loss: 2.6790 - val_accuracy: 0.2011
2020-09-07

**<font color='blue'>본 스크립트는 SageMaker상의 notebook에서 구동하고 있지만, 여러분의 로컬 컴퓨터에서도 python과 jupyter notebook이 정상적으로 인스톨되어 있다면 동일하게 수행 가능합니다.</font>**

## Use TensorFlow Script Mode

TensorFlow 버전 1.11 이상에서 Amazon SageMaker Python SDK는 **스크립트 모드(Script mode)**를 지원합니다. 스크립트 모드는 종래 레거시 모드(Legacy mode) 대비 아래 장점들이 있습니다.

* 스크립트 모드의 학습 스크립트는 일반적으로 TensorFlow 용으로 작성하는 학습 스크립트와 더 유사하므로 TensorFlow 학습 스크립트를 최소한의 변경으로 실행할 수 있습니다. 따라서, 기존 레거시 모드보다 TensorFlow 학습 스크립트를 수정하는 것이 더 쉽습니다. 
    - 레거시 모드는 Tensorflow Estimator API를 기반으로 한 아래의 함수들을 반드시 포함해야 합니다.
        - 아래 함수들에서 하나의 함수를 만드시 포함해야 합니다.
            - `model_fn`: 학습할 모델을 정의합니다,
            - `keras_model_fn`: 학습할 tf.keras 모델을 정의합니다.
            - `estimator_fn`: 학습할 tf.estimator.Estimator를 정의합니다.
        - `train_input_fn`: 학습 데이터 로딩과 전처리를 수행합니다. 
        - `eval_input_fn`: 검증 데이터의 로딩과 전처리를 수행합니다.
        - (Optional) `serving_input_fn`: 예측(prediction) 중에 모델에 전달할 feautre를 정의합니다. 이 함수는 학습시에만 사용되지만, SageMaker 엔드포인트에서 모델을 배포할 때 필요합니다.
    - `if __name__ == “__main__”:` 블록을 정의할 수 없어 디버깅이 쉽지 않습니다.
    
* 스크립트 모드는 Python 2.7-와 Python 3.6-을 지원합니다.

* 스크립트 모드는 **Hovorod 기반 분산 학습(distributed training)도 지원**합니다.

TensorFlow 스크립트 모드에서 학습 스크립트를 작성하는 방법 및 Tensorflow 스크립트 모드의 estimator와 model 사용법에 대한 자세한 내용은
https://sagemaker.readthedocs.io/en/stable/using_tf.html 을 참조하세요.

### Preparing your script for training in SageMaker

SageMaker 스크립트 모드의 학습 스크립트는 SageMaker 외부에서 실행할 수 있는 학습 스크립트와 매우 유사합니다.
SageMaker는 하나의 인자값(argument), model_dir와 로그 및 모델 아티팩트(model artifacts)에 사용되는 S3 경로로 학습 스크립트를 실행합니다.

SageMaker 학습 인스턴스에서는 학습의 컨테이너에 S3에 저장된 데이터를 다운로드하여 학습에 활용합니다. 그 때, S3 버킷의 데이터 경로와 컨테이너의 데이터 경로를 컨테이너 환경 변수를 통해 연결합니다.

여러분은 다양한 환경 변수를 통해 학습 환경에 대한 유용한 속성들(properties)에 액세스할 수 있습니다.
이 스크립트의 경우 `Train, Validation, Eval`이라는 3 개의 데이터 채널을 스크립트로 보냅니다.

----
### TODO 1.
`cifar10_keras_sm_tf2.py`파일에서 SageMaker API 환경 변수 SM_CHANNEL_TRAIN, SM_CHANNEL_VALIDATION, SM_CHANNEL_EVAL에서 디폴트 값을 가져오기 위해 train, validation, eval 인수를 수정해 주세요. 

`cifar10_keras_sm_tf2.py`의 `if __name__ == '__main__':` 블록 내에 아래 인자값을 수정해 주세요.

```python
parser.add_argument(
        '--train',
        type=str,
        required=False,
        default=os.environ.get('SM_CHANNEL_TRAIN'), # <-- 수정 부분
        help='The directory where the CIFAR-10 input data is stored.')
parser.add_argument(
        '--validation',
        type=str,
        required=False,
        default=os.environ.get('SM_CHANNEL_VALIDATION'), # <-- 수정 부분
        help='The directory where the CIFAR-10 input data is stored.')
parser.add_argument(
        '--eval',
        type=str,
        required=False,
        default=os.environ.get('SM_CHANNEL_EVAL'), # <-- 수정 부분
        help='The directory where the CIFAR-10 input data is stored.')
```


환경 변수에 따른 S3 경로와 컨테이너 경로는 아래 표와 같습니다.

|  S3 경로  |  환경 변수  |  컨테이너 경로  |
| :---- | :---- | :----| 
|  s3://bucket_name/prefix/train  |  `SM_CHANNEL_TRAIN`  | `/opt/ml/input/data/train`  |
|  s3://bucket_name/prefix/validation  |  `SM_CHANNEL_VALIDATION`  | `/opt/ml/input/data/validation`  |
|  s3://bucket_name/prefix/eval  |  `SM_CHANNEL_EVAL`  | `/opt/ml/input/data/eval`  |
|  s3://bucket_name/prefix/model.tar.gz  |  `SM_MODEL_DIR`  |  `/opt/ml/model`  |
|  s3://bucket_name/prefix/output.tar.gz  |  `SM_OUTPUT_DATA_DIR`  |  `/opt/ml/output/data`  |

얘를 들어, `/opt/ml/input/data/train`은 학습 데이터가 다운로드되는 컨테이너 내부의 디렉토리입니다.

자세한 내용은 아래의 SageMaker Python SDK 문서를 확인하시기 바랍니다.<br>
(https://sagemaker.readthedocs.io/en/stable/using_tf.html#preparing-a-script-mode-training-script)


SageMaker는 train, validation, eval 경로들을 직접 인자로 보내지 않고, 대신 스크립트에서 환경 변수를 사용하여 해당 인자를 필요하지 않은 것으로 표시합니다.

SageMaker는 유용한 환경 변수를 여러분이 작성한 학습 스크립트로 보냅니다. 예시들은 아래와 같습니다.
* `SM_MODEL_DIR`: 학습 작업이 모델 아티팩트(model artifacts)를 저장할 수 있는 로컬 경로를 나타내는 문자열입니다. 학습 완료 후, 해당 경로 내 모델 아티팩트는 모델 호스팅을 위해 S3에 업로드됩니다. 이는 S3 위치인 학습 스크립트에 전달 된 model_dir 인수와 다르다는 점을 주의해 주세요. SM_MODEL_DIR은 항상 `/opt/ml/model`로 설정됩니다.
* `SM_NUM_GPUS`: 호스트(Host)에서 사용 가능한 GPU 수를 나타내는 정수(integer)입니다.
* `SM_OUTPUT_DATA_DIR`: 출력 아티팩트를 저장할 디렉토리의 경로를 나타내는 문자열입니다. 출력 아티팩트에는 체크포인트, 그래프 및 다른 저장용 파일들이 포함될 수 있지만 모델 아티팩트는 포함되지 않습니다. 이 출력 아티팩트들은 압축되어 모델 아티팩트와 동일한 접두사가 있는 S3 버킷으로 S3에 업로드됩니다.

이 샘플 코드는 네트워크 지연을 줄이기 위해 모델의 체크포인트(checkpoints)를 로컬 환경에 저장합니다. 이들은 학습 종료 후 S3에 업로드할 수 있습니다.

----
### TODO 2.

`cifar10_keras_sm_tf2.py`의 `if __name__ == '__main__':` 블록 내에 아래 인자값을 추가해 주세요.

```python
parser.add_argument(
        '--model_output_dir',
        type=str,
        default=os.environ.get('SM_MODEL_DIR'))
```

----
### TODO 3.
`ModelCheckpoint` 함수의 저장 경로를 새 경로로 아래와 같이 수정해 주세요.

From:
```python
callbacks.append(ModelCheckpoint(args.model_dir + '/checkpoint-{epoch}.h5'))
```
To:
```python
callbacks.append(ModelCheckpoint(args.model_output_dir + '/checkpoint-{epoch}.h5'))
```

----
### TODO 4.
`save_model` 함수의 인자값을 아래와 같이 수정해 주세요.

From:  
```python
return save_model(model, args.model_dir)
```
To:  
```python
return save_model(model, args.model_output_dir)
```
S

### Test your script locally (just like on your laptop)

테스트를 위해 위와 동일한 명령(command)으로 새 스크립트를 실행하고, 예상대로 실행되는지 확인합니다. <br>
SageMaker TensorFlow API 호출 시에 환경 변수들은 자동으로 넘겨기지만, 로컬 주피터 노트북에서 테스트 시에는 수동으로 환경 변수들을 지정해야 합니다. (아래 예제 코드를 참조해 주세요.)

```python
%env SM_MODEL_DIR=./logs
```

In [6]:
%%time
!mkdir -p logs   

# Number of GPUs on this machine
%env SM_NUM_GPUS=1
# Where to save the model
%env SM_MODEL_DIR=./logs
# Where the training data is
%env SM_CHANNEL_TRAIN=data/train
# Where the validation data is
%env SM_CHANNEL_VALIDATION=data/validation
# Where the evaluation data is
%env SM_CHANNEL_EVAL=data/eval


!python training_script/cifar10_keras_sm_tf2.py --model_dir ./logs --epochs 1
!rm -rf logs

env: SM_NUM_GPUS=1
env: SM_MODEL_DIR=./logs
env: SM_CHANNEL_TRAIN=data/train
env: SM_CHANNEL_VALIDATION=data/validation
env: SM_CHANNEL_EVAL=data/eval
2020-09-07 12:25:03.124864: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX512F
2020-09-07 12:25:03.148227: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2999995000 Hz
2020-09-07 12:25:03.148598: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x564aa9010fe0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-09-07 12:25:03.148614: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2020-09-07 12:25:03.148715: I tensorflow/core/common_runtime/process_util.cc:147] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
Train for 312 steps, validate for 78

### Use SageMaker local for local testing

본격적으로 학습을 시작하기 전에 로컬 모드를 사용하여 디버깅을 먼저 수행합니다. 로컬 모드는 학습 인스턴스를 생성하는 과정이 없이 로컬 인스턴스로 컨테이너를 가져온 후 곧바로 학습을 수행하기 때문에 코드를 보다 신속히 검증할 수 있습니다.

Amazon SageMaker Python SDK의 로컬 모드는 TensorFlow 또는 MXNet estimator서 단일 인자값을 변경하여 CPU (단일 및 다중 인스턴스) 및 GPU (단일 인스턴스) SageMaker 학습 작업을 에뮬레이션(enumlate)할 수 있습니다. 

로컬 모드 학습을 위해서는 docker-compose 또는 nvidia-docker-compose (GPU 인스턴스인 경우)의 설치가 필요합니다. 아래 코드 셀을 통해 본 노트북 환경에 docker-compose 또는 nvidia-docker-compose를 설치하고 구성합니다. 
 
로컬 모드의 학습을 통해 여러분의 코드가 현재 사용 중인 하드웨어를 적절히 활용하고 있는지 확인하기 위한 GPU 점유와 같은 지표(metric)를 쉽게 모니터링할 수 있습니다.

In [5]:
import os
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

role = get_execution_role()

In [7]:
from sagemaker.tensorflow import TensorFlow
estimator = TensorFlow(base_job_name='cifar10',
                       entry_point='cifar10_keras_sm_tf2.py',
                       source_dir='training_script',
                       role=role,
                       framework_version='2.1.0',
                       py_version='py3',
                       script_mode=True,
                       hyperparameters={'epochs' : 1},
                       train_instance_count=1, 
                       train_instance_type='local')

학습을 수행할 3개의 채널과 데이터의 경로를 지정합니다. **로컬 모드로 수행하기 때문에 S3 경로 대신 노트북 인스턴스의 경로를 지정하시면 됩니다.**

In [8]:
%%time
estimator.fit({'train': 'file://data/train',
               'validation': 'file://data/validation',
               'eval': 'file://data/eval'})

'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


Creating tmpzu25y72b_algo-1-izwuh_1 ... 
Attaching to tmpzu25y72b_algo-1-izwuh_12mdone
algo-1-izwuh_1  | 2020-09-07 12:27:13,940 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
algo-1-izwuh_1  | 2020-09-07 12:27:13,947 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-izwuh_1  | 2020-09-07 12:27:15,112 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-izwuh_1  | 2020-09-07 12:27:15,126 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-izwuh_1  | 2020-09-07 12:27:15,138 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-izwuh_1  | 2020-09-07 12:27:15,147 sagemaker-containers INFO     Invoking user script
algo-1-izwuh_1  | 
algo-1-izwuh_1  | Training Env:
algo-1-izwuh_1  | 
algo-1-izwuh_1  | {
algo-1-izwuh_1  |     "additional_framework_parameters": {},
algo-1-izwuh_1  |     "channel_input_dirs": {
algo-1-izwuh_1  

Estimator가 처음 실행될 때 Amazon ECR 리포지토리(repository)에서 컨테이너 이미지를 다운로드해야 하지만 학습을 즉시 시작할 수 있습니다. 즉, 별도의 학습 클러스터가 프로비저닝 될 때까지 기다릴 필요가 없습니다. 또한 반복 및 테스트시 필요할 수 있는 후속 실행에서 MXNet 또는 TensorFlow 스크립트에 대한 수정 사항이 즉시 실행되기 시작합니

### Dependency 설정 및 Laucher code 작성하기

* Container에 없는 파일을 추가적으로 설치하거나 dependency code를 패스 하기 귀해 다음과 launcher code를 사용할 수 있습니다.
  이 예제에서는 ```" cifar10_keras_sm_tf2_dependency.py"``` 에 ```import jsonlines```를 추가하였고 프로그램 시작 전에 requirment.txt에 페키지를 pip로 설치하도록 합니다.  
* Requirements.txt를 위의 Python 실행 코드와 함께 동일 폴더에 저장합니다. 
* Script모드에서 AWS Deeplearning container에 requirements.txt 에 정의된 패키를 설치한 후 실행하게 됩니다.
* 참고: https://sagemaker.readthedocs.io/en/stable/frameworks/pytorch/using_pytorch.html?highlight=requirements#id14 

In [9]:
data_dir = os.path.join(os.getcwd(), 'training_script')
os.makedirs(data_dir, exist_ok=True)
data_dir

'/home/ec2-user/SageMaker/sagemaker-tf-examples/training_script'

In [10]:
!wget -q https://raw.githubusercontent.com/aws-samples/amazon-sagemaker-script-mode/master/local_mode_setup.sh
!wget -q https://raw.githubusercontent.com/aws-samples/amazon-sagemaker-script-mode/master/daemon.json    
!/bin/bash ./local_mode_setup.sh

SageMaker instance route table setup is ok. We are good to go.
SageMaker instance routing for Docker is ok. We are good to go!


In [11]:
%%writefile training_script/launcher.sh

/usr/bin/python3 -m pip install --upgrade pip
pip install -r requirements.txt


python cifar10_keras_sm_tf2_dependency.py --model_dir ${SM_MODEL_DIR} --epochs ${SM_HP_EPOCHS}

echo "Generated image $(ls ${SM_MODEL_DIR})"



Overwriting training_script/launcher.sh


In [12]:
!cat training_script/launcher.sh


/usr/bin/python3 -m pip install --upgrade pip
pip install -r requirements.txt


python cifar10_keras_sm_tf2_dependency.py --model_dir ${SM_MODEL_DIR} --epochs ${SM_HP_EPOCHS}

echo "Generated image $(ls ${SM_MODEL_DIR})"



In [17]:
train_instance_type='local'

이제 requirement.txt가 시작 전에 설치 되고 시작되었는지 확인해 봅니다. 예제에서는 requiremet.txt에 dummy로 jsonlines를 import하고 Jsonline object에서 1,2,3을 프린트 하도록 해 놓았습니다. 

In [21]:
import sagemaker
from sagemaker.tensorflow import TensorFlow

estimator = TensorFlow(base_job_name='cifar10',
                       entry_point='launcher.sh',
                       dependencies=['training_script/cifar10_keras_sm_tf2_dependency.py'],
                       source_dir=data_dir,
                       train_instance_type='local',
                       train_instance_count=1,
                       role=role,
                       hyperparameters={'epochs' : 1},
                       framework_version='2.1.0',
                       py_version='py3',
                       script_mode=True)


In [22]:
%%time
estimator.fit({'train': 'file://data/train',
               'validation': 'file://data/validation',
               'eval': 'file://data/eval'
               })

'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


Creating tmp74v1wg1h_algo-1-4qwuk_1 ... 
Attaching to tmp74v1wg1h_algo-1-4qwuk_12mdone
algo-1-4qwuk_1  | 2020-09-07 12:32:54,663 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
algo-1-4qwuk_1  | 2020-09-07 12:32:54,669 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-4qwuk_1  | 2020-09-07 12:32:55,840 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-4qwuk_1  | 2020-09-07 12:32:55,854 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-4qwuk_1  | 2020-09-07 12:32:55,867 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-4qwuk_1  | 2020-09-07 12:32:55,876 sagemaker-containers INFO     Invoking user script
algo-1-4qwuk_1  | 
algo-1-4qwuk_1  | Training Env:
algo-1-4qwuk_1  | 
algo-1-4qwuk_1  | {
algo-1-4qwuk_1  |     "additional_framework_parameters": {},
algo-1-4qwuk_1  |     "channel_input_dirs": {
algo-1-4qwuk_1  

### Using SageMaker for faster training time

이번에는 로컬 모드를 사용하지 않고 SageMaker 학습에 GPU 학습 인스턴스를 생성하여 학습 시간을 단축해 봅니다.<br>
로컬 모드와 다른 점들은 (1) `train_instance_type`이 로컬 모드의 ‘local’ 대신 여러분이 원하는 특정 인스턴스 유형으로 설정해야 하고, (2) 학습 데이터를 Amazon S3에 업로드 후 학습 경로를 S3 경로로 설정해야 합니다. 

SageMaker SDK는 S3 업로드를 위한 간단한 함수(`Session.upload_data()`)를 제공합니다. 이 함수를 통해 리턴되는 값은 데이터가 저장된 S3 경로입니다.
좀 더 자세한 설정이 필요하다면 SageMaker SDK 대신 boto3를 사용하시면 됩니다.

*[Note]: 고성능 워크로드를 위해 Amazon EFS와 Amazon FSx for Lustre도 지원하고 있습니다. 자세한 정보는 아래의 AWS 블로그를 참조해 주세요.<br>
https://aws.amazon.com/blogs/machine-learning/speed-up-training-on-amazon-sagemaker-using-amazon-efs-or-amazon-fsx-for-lustre-file-systems/*

In [23]:
dataset_location = sagemaker_session.upload_data(path='data', key_prefix='data/DEMO-cifar10')
display(dataset_location)

's3://sagemaker-us-east-2-870180618679/data/DEMO-cifar10'

S3에 데이터 업로드를 완료했다면, Estimator를 새로 생성합니다. <br>
아래 코드를 그대로 복사 후에 `train_instance_type='local'`을 `train_instance_type='ml.p2.xlarge'`로 수정하고
`hyperparameters={'epochs': 1}`를 `hyperparameters={'epochs': 5}`로 수정합니다.

```python
from sagemaker.tensorflow import TensorFlow
estimator = TensorFlow(base_job_name='cifar10',
                       entry_point='cifar10_keras_sm_tf2.py',
                       source_dir='training_script',
                       role=role,
                       framework_version='2.0.0',
                       py_version='py3',
                       script_mode=True,                       
                       hyperparameters={'epochs': 1},
                       train_instance_count=1, 
                       train_instance_type='local')
```

*[Note] 
2019년 8월부터 SageMaker에서도 학습 인스턴스에 EC2 spot instance를 사용하여 비용을 크게 절감할 수 있습니다. 자세한 정보는 아래의 AWS 블로그를 참조해 주세요.<br>
https://aws.amazon.com/ko/blogs/korea/managed-spot-training-save-up-to-90-on-your-amazon-sagemaker-training-jobs/*

만약 Managed Spot Instance로 학습하려면 다음 코드를 Estimator의 train_instance_type의 다음 행에 추가해 주세요.
```python
train_max_run = 3600,
train_use_spot_instances = 'True',
train_max_wait = 3600,
```

In [24]:
from sagemaker.tensorflow import TensorFlow
estimator = TensorFlow(base_job_name='cifar10',
                       entry_point='cifar10_keras_sm_tf2.py',
                       source_dir=data_dir,
                       role=role,
                       framework_version='2.1.0',
                       py_version='py3',
                       script_mode=True,                       
                       hyperparameters={'epochs': 5},
                       train_instance_count=1, 
                       train_instance_type='ml.p3.2xlarge',
                       train_max_run = 3600,
                       train_use_spot_instances = 'True',
                       train_max_wait = 3600,
                       )

학습을 수행합니다. 이번에는 각각의 채널(`train, validation, eval`)에 S3의 데이터 저장 위치를 지정합니다.<br>
학습 완료 후 Billable seconds도 확인해 보세요. Billable seconds는 실제로 학습 수행 시 과금되는 시간입니다.
```
Billable seconds: <time>
```

참고로, `ml.p2.xlarge` 인스턴스로 5 epoch 학습 시 전체 6분-7분이 소요되고, 실제 학습에 소요되는 시간은 3분-4분이 소요됩니다.

In [25]:
%%time
estimator.fit({'train':'{}/train'.format(dataset_location),
              'validation':'{}/validation'.format(dataset_location),
              'eval':'{}/eval'.format(dataset_location)})

's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


2020-09-07 12:34:34 Starting - Starting the training job...
2020-09-07 12:34:36 Starting - Launching requested ML instances......
2020-09-07 12:35:40 Starting - Preparing the instances for training......
2020-09-07 12:36:47 Downloading - Downloading input data...
2020-09-07 12:37:06 Training - Downloading the training image.....
2020-09-07 12:38:10 Training - Training image download completed. Training in progress.2020-09-07 12:38:14,266 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
2020-09-07 12:38:14,672 sagemaker-containers INFO     Invoking user script

Training Env:

{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "eval": "/opt/ml/input/data/eval",
        "validation": "/opt/ml/input/data/validation",
        "train": "/opt/ml/input/data/train"
    },
    "current_host": "algo-1",
    "framework_module": "sagemaker_tensorflow_container.training:main",
    "hosts": [
        "algo-1"
    ],
    "hyperparam

## Start a new SageMaker experiment

Amazon SageMaker Experiments는 데이타 과학자들이 머신 러닝 실험을 구성하고, 추적하고, 비교하고, 평가할 수 있게 합니다.
머신 러닝은 반복적인 과정 입니다. 데이타 과학자들은 증분적인 모델 정확도의 변화를 관찰하면서, 데이타, 알고리즘, 파라미터의 조합들을 가지고 실험을 할 필요가 있습니다. 이러한 반복적인 과정은 수 많은 모델 훈련 및 모델의 버전들을 가지게 됩니다. 이것은 성능이 좋은 모델들 및 입력 설정의 구성들을 추적하기가 어렵게 됩니다. 이것은 더욱 더 증분적인 향상을 위한 기회를 찾기 위해서, 현재의 실험들과 과거에 수행한 실험들의 비교를 더욱 더 어렵게 합니다. 

**Amazon SageMaker Experiments는 반복적인 과정(시험, Trial)으로서의 입력 값들, 파라미터들, 구성 설정 값들 및 결과들을 자동으로 추적 할 수 있게 합니다.<br>
데이타 과학자들은 시험들(Trials)을 실험(Experiment) 안으로 할당하고, 그룹핑하고, 구성할 수 있습니다.**
Amazon SageMaker Experiments는 현재 및 과거의 실험들을 시각적으로 조회할 수 있게 하는 Amazon SageMaker Studio와 통합이 되어 있습니다. Amazon SageMaker Studio는 또한 주요 평가 지표를 가지고 시험들을 비교할 수 있으며, 가장 우수한 모델들을 확인할 수 있게 합니다.  


`sagemaker-experiments` 패키지를 먼저 설치합니다.

In [26]:
!pip install sagemaker-experiments

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/tensorflow2_p36/bin/python -m pip install --upgrade pip' command.


이제 실험(Experiment)을 만듭니다.

In [41]:
from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
import time

# Create an aexperiment
cifar10_experiment = Experiment.create(
    experiment_name="TensorFlow-cifar10-experiment",
    description="Classification of cifar10 images")

In [42]:
print(cifar10_experiment)

Experiment(sagemaker_boto_client=<botocore.client.SageMaker object at 0x7efc6964db00>,experiment_name='TensorFlow-cifar10-experiment',description='Classification of cifar10 images',tags=None,experiment_arn='arn:aws:sagemaker:us-east-2:870180618679:experiment/tensorflow-cifar10-experiment',response_metadata={'RequestId': 'cdff9d0e-53fc-4e0b-9dd7-e22498df865a', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'cdff9d0e-53fc-4e0b-9dd7-e22498df865a', 'content-type': 'application/x-amz-json-1.1', 'content-length': '101', 'date': 'Mon, 07 Sep 2020 12:53:24 GMT'}, 'RetryAttempts': 0})


다음은 시험(Trial)을 생성 합니다. 이 시험은 GPU Instance 위에서 Epoch 5를 가지고 실행하게 됩니다.

In [44]:
# Create a trial
trial_name = f"cifar10-training-job-{int(time.time())}"
trial = Trial.create(
    trial_name=trial_name, 
    experiment_name=cifar10_experiment.experiment_name
)

새로운 estimator를 생성 합니다.

In [45]:
from sagemaker.tensorflow import TensorFlow
estimator = TensorFlow(base_job_name='cifar10',
                       entry_point='cifar10_keras_sm_tf2.py',
                       source_dir=data_dir,
                       role=role,
                       framework_version='2.1.0',
                       py_version='py3',
                       script_mode=True,                       
                       hyperparameters={'epochs': 5},
                       train_instance_count=1, 
                       train_instance_type='ml.p3.2xlarge'
                       )

다음은 각각 입력 데이타의 채널에 대한 S3 data location을 사용합니다.
```python
dataset_location + '/train'
dataset_location + '/validation' 
dataset_location + '/eval'
```
위에서 설정한 experiment config를 fit 함수의 파라미터로 추가합니다. 또한 시험은 훈련 Job과 연결이 됩니다.
<br>TrialComponent는 시험(Trail)의 한 요소를 의미합니다. 여기서는 "Training"의 훈련 요소를 지칭합니다.
```python
experiment_config={
                  "ExperimentName": cifar10_experiment.experiment_name, 
                  "TrialName": trial.trial_name,
                  "TrialComponentDisplayName": "Training"}
```

In [46]:
estimator.fit({'train' :  dataset_location + '/train',
               'validation' :  dataset_location + '/validation',
               'eval' :  dataset_location + '/eval'
              },
              experiment_config={
                "ExperimentName": cifar10_experiment.experiment_name, 
                "TrialName": trial.trial_name,
                "TrialComponentDisplayName": "Training"
              }
            )

INFO:sagemaker:Creating training-job with name: cifar10-2020-09-07-12-54-28-596


2020-09-07 12:54:28 Starting - Starting the training job...
2020-09-07 12:54:31 Starting - Launching requested ML instances......
2020-09-07 12:55:33 Starting - Preparing the instances for training...
2020-09-07 12:56:26 Downloading - Downloading input data
2020-09-07 12:56:26 Training - Downloading the training image.........
2020-09-07 12:57:49 Training - Training image download completed. Training in progress..2020-09-07 12:57:53,164 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
2020-09-07 12:57:53,553 sagemaker-containers INFO     Invoking user script

Training Env:

{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "eval": "/opt/ml/input/data/eval",
        "validation": "/opt/ml/input/data/validation",
        "train": "/opt/ml/input/data/train"
    },
    "current_host": "algo-1",
    "framework_module": "sagemaker_tensorflow_container.training:main",
    "hosts": [
        "algo-1"
    ],
    "hyperparame

## Analyze the experiments

여기서는 DisplayName 이 "Training"과 같은 시험 요소(Trial Component)만 찾는 필터를 생성합니다.
위에서 설정한 TrialComponentDisplayName": "Training" 을 찾게 됩니다.

In [47]:
search_expression = {
    "Filters":[
        {
            "Name": "DisplayName",
            "Operator": "Equals",
            "Value": "Training",
        }
    ],
}

ExperimentAnalytics 함수에 experiment 이름과 위에서 생성한 필터를 파라미터로 제공합니다.

In [48]:
import pandas as pd 
pd.options.display.max_columns = 500

from sagemaker.analytics import ExperimentAnalytics
trial_component_analytics = ExperimentAnalytics(
    sagemaker_session=sagemaker_session, 
    experiment_name=cifar10_experiment.experiment_name,
    search_expression=search_expression
)

table = trial_component_analytics.dataframe(force_refresh=True)
display(table)

,TrialComponentName,DisplayName,SourceArn,SageMaker.ImageUri,SageMaker.InstanceCount,SageMaker.InstanceType,SageMaker.VolumeSizeInGB,epochs,model_dir,sagemaker_container_log_level,sagemaker_enable_cloudwatch_metrics,sagemaker_job_name,sagemaker_program,sagemaker_region,sagemaker_submit_directory,val_accuracy_EVAL - Min,val_accuracy_EVAL - Max,val_accuracy_EVAL - Avg,val_accuracy_EVAL - StdDev,val_accuracy_EVAL - Last,val_accuracy_EVAL - Count,loss_TRAIN - Min,loss_TRAIN - Max,loss_TRAIN - Avg,loss_TRAIN - StdDev,loss_TRAIN - Last,loss_TRAIN - Count,batch_TRAIN - Min,batch_TRAIN - Max,batch_TRAIN - Avg,batch_TRAIN - StdDev,batch_TRAIN - Last,batch_TRAIN - Count,accuracy_EVAL - Min,accuracy_EVAL - Max,accuracy_EVAL - Avg,accuracy_EVAL - StdDev,accuracy_EVAL - Last,accuracy_EVAL - Count,accuracy_TRAIN - Min,accuracy_TRAIN - Max,accuracy_TRAIN - Avg,accuracy_TRAIN - StdDev,accuracy_TRAIN - Last,accuracy_TRAIN - Count,size_EVAL - Min,size_EVAL - Max,size_EVAL - Avg,size_EVAL - StdDev,size_EVAL - Last,size_EVAL - Count,size_TRAIN - Min,size_TRAIN - Max,size_TRAIN - Avg,size_TRAIN - StdDev,size_TRAIN - Last,size_TRAIN - Count,val_loss_EVAL - Min,val_loss_EVAL - Max,val_loss_EVAL - Avg,val_loss_EVAL - StdDev,val_loss_EVAL - Last,val_loss_EVAL - Count,batch_EVAL - Min,batch_EVAL - Max,batch_EVAL - Avg,batch_EVAL - StdDev,batch_EVAL - Last,batch_EVAL - Count,loss_EVAL - Min,loss_EVAL - Max,loss_EVAL - Avg,loss_EVAL - StdDev,loss_EVAL - Last,loss_EVAL - Count,eval - MediaType,eval - Value,train - MediaType,train - Value,validation - MediaType,validation - Value,SageMaker.DebugHookOutput - MediaType,SageMaker.DebugHookOutput - Value,SageMaker.ModelArtifact - MediaType,SageMaker.ModelArtifact - Value
0,cifar10-2020-09-07-12-54-28-596-aws-training-job,Training,arn:aws:sagemaker:us-east-2:870180618679:train...,763104351884.dkr.ecr.us-east-2.amazonaws.com/t...,1.0,ml.p3.2xlarge,30.0,5.0,"""s3://sagemaker-us-east-2-870180618679/cifar10...",20.0,false,"""cifar10-2020-09-07-12-54-28-596""","""cifar10_keras_sm_tf2.py""","""us-east-2""","""s3://sagemaker-us-east-2-870180618679/cifar10...",0.357873,0.619792,0.479067,0.114224,0.619792,5,1.014336,3.854164,1.885697,1.334685,1.104554,4,0.0,252.0,126.0,114.658914,252.0,4,0.319737,0.620843,0.478061,0.125329,0.620843,6,0.085938,0.618145,0.430075,0.241016,0.618145,4,128.0,128.0,128.0,0.0,128.0,1,128.0,128.0,128.0,0.0,128.0,4,1.061726,1.798096,1.503884,0.360311,1.061726,5,0.0,4.0,1.666667,1.632993,4.0,6,1.066103,1.855191,1.413432,0.303578,1.066103,6,None,s3://sagemaker-us-east-2-870180618679/data/DEM...,None,s3://sagemaker-us-east-2-870180618679/data/DEM...,None,s3://sagemaker-us-east-2-870180618679/data/DEM...,None,s3://sagemaker-us-east-2-870180618679/,None,s3://sagemaker-us-east-2-870180618679/cifar10-...


### Clean up the Experiment
experiment 이름은 계정과 리젼에 유니크한 이름이기에, 사용을 하지 않는다면 지워주는 것이 좋습니다.<br>
위에서 생성한 cifar10_experiment 오브젝트를 아래 cleanup 함수에 파라미터로 주어서 지워주게 됩니다.
이 작업은 관련된 Trial Component, Trial 을 지우고, 마지막으로 experiment를 삭제합니다.

In [49]:
import boto3

sess = boto3.Session()
sm = sess.client('sagemaker')
from smexperiments.trial_component import TrialComponent

def cleanup(experiment):
    for trial_summary in experiment.list_trials():
        trial = Trial.load(sagemaker_boto_client=sm, trial_name=trial_summary.trial_name)
        for trial_component_summary in trial.list_trial_components():
            tc = TrialComponent.load(
                sagemaker_boto_client=sm,
                trial_component_name=trial_component_summary.trial_component_name)
            trial.remove_trial_component(tc)
            try:
                # comment out to keep trial components
                tc.delete()
            except:
                # tc is associated with another trial
                continue
            # to prevent throttling
            time.sleep(.5)
        trial.delete()
    experiment.delete()
    print("The experiemnt is deleted")


cleanup(cifar10_experiment)    

The experiemnt is deleted


수고하셨습니다.

SageMaker에서 GPU 인스턴스를 사용해 5 epoch를 정상적으로 학습할 수 있었습니다.
다음 노트북으로 계속 진행하기 전에 SageMaker 콘솔의 Training jobs 섹션을 살펴보고 여러분이 수행한 job을 찾아 configuration을 확인하세요.

스크립트 모드 학습에 대한 자세한 내용은 아래의 AWS 블로그를 참조해 주세요.
Using TensorFlow eager execution with Amazon SageMaker script mode


In [51]:
#!aws sagemaker list-experiments
#!aws sagemaker list-trials

#!aws sagemaker delete-trial --trial-name cifar10-training-job-1599480857
#!aws sagemaker delete-experiment --experiment-name TensorFlow-cifar10-experiment

